# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import numpy as np
import pandas as pd
import pickle
from pprint import pprint
import re
import sys
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import precision_recall_fscore_support
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sqlalchemy import create_engine
import time
import warnings
warnings.filterwarnings('ignore')

In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterMessages.db')
df = pd.read_sql_table("DisasterMessages", con=engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
X = df['message']
Y = df.iloc[:, 4:]

In [4]:
df['related'].value_counts()

1    19906
0     6122
2      188
Name: related, dtype: int64

In [5]:
Y.shape

(26216, 36)

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    
    # normalize text
    text = re.sub(r"[^\w]", " ", text.lower())
    
    # tokenize text
    words = word_tokenize(text)
    
    # remove stopwords
    stopwords_ = stopwords.words("english")
    words = [word for word in words if word not in stopwords_]
    
    # extract root form of words
    words = [WordNetLemmatizer().lemmatize(word, pos='v') for word in words]
    
    return words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(OneVsRestClassifier(LinearSVC())))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_test, X_train, Y_test, Y_train = train_test_split(X, Y)
# train classifier
pipeline.fit(X_train, Y_train)

# predict on test data
Y_pred = pipeline.predict(X_test)

In [9]:
Y_pred

array([[1, 1, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
for col in range(36):
    print(Y_test.columns[col])
    print(classification_report(Y_test.iloc[:,col], Y_pred[:,col]))
    print('-----------------------------------------------------')


related
              precision    recall  f1-score   support

           0       0.63      0.46      0.54      4582
           1       0.84      0.92      0.88     14936
           2       0.70      0.15      0.24       144

    accuracy                           0.81     19662
   macro avg       0.72      0.51      0.55     19662
weighted avg       0.79      0.81      0.79     19662

-----------------------------------------------------
request
              precision    recall  f1-score   support

           0       0.91      0.96      0.94     16288
           1       0.75      0.55      0.63      3374

    accuracy                           0.89     19662
   macro avg       0.83      0.75      0.78     19662
weighted avg       0.88      0.89      0.88     19662

-----------------------------------------------------
offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     19570
           1       0.00      0.00      0.00        92

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     19577
           1       0.00      0.00      0.00        85

    accuracy                           1.00     19662
   macro avg       0.50      0.50      0.50     19662
weighted avg       0.99      1.00      0.99     19662

-----------------------------------------------------
aid_centers
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     19427
           1       0.00      0.00      0.00       235

    accuracy                           0.99     19662
   macro avg       0.49      0.50      0.50     19662
weighted avg       0.98      0.99      0.98     19662

-----------------------------------------------------
other_infrastructure
              precision    recall  f1-score   support

           0       0.96      1.00      0.98     18781
           1       0.29      0.04      0.07       881

    accuracy                           0.

In [11]:
print('Accuracy: {}'.format(np.mean(Y_test.values == Y_pred)))

Accuracy: 0.9465409871268888


### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
parameters = {'vect__ngram_range': ((1, 1), (1, 2)),
                  'vect__max_df': (0.75, 1.0)
                  }
model = GridSearchCV(estimator=pipeline, param_grid=parameters, cv=5)

model


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        prep

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [13]:
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)

In [14]:
for col in range(36):
    print(Y_test.columns[col])
    print(classification_report(Y_test.iloc[:,col], Y_pred[:,col]))
    print('-----------------------------------------------------')

related
              precision    recall  f1-score   support

           0       0.69      0.41      0.51      4582
           1       0.83      0.94      0.88     14936
           2       0.62      0.09      0.16       144

    accuracy                           0.81     19662
   macro avg       0.71      0.48      0.52     19662
weighted avg       0.80      0.81      0.79     19662

-----------------------------------------------------
request
              precision    recall  f1-score   support

           0       0.92      0.96      0.94     16288
           1       0.74      0.61      0.67      3374

    accuracy                           0.90     19662
   macro avg       0.83      0.78      0.80     19662
weighted avg       0.89      0.90      0.89     19662

-----------------------------------------------------
offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     19570
           1       0.00      0.00      0.00        92

              precision    recall  f1-score   support

           0       0.96      1.00      0.98     18781
           1       0.37      0.03      0.06       881

    accuracy                           0.95     19662
   macro avg       0.66      0.51      0.52     19662
weighted avg       0.93      0.95      0.94     19662

-----------------------------------------------------
weather_related
              precision    recall  f1-score   support

           0       0.89      0.92      0.91     14177
           1       0.78      0.72      0.75      5485

    accuracy                           0.87     19662
   macro avg       0.84      0.82      0.83     19662
weighted avg       0.86      0.87      0.86     19662

-----------------------------------------------------
floods
              precision    recall  f1-score   support

           0       0.96      0.99      0.98     18054
           1       0.86      0.58      0.69      1608

    accuracy                           0.96     196

In [15]:
print('Accuracy: {}'.format(np.mean(Y_test.values == Y_pred)))

Accuracy: 0.9482193514845331


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [16]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))])

In [17]:
# train classifier
pipeline.fit(X_train, Y_train)

# predict on test data
Y_pred = pipeline.predict(X_test)

In [18]:
for col in range(36):
    print(Y_test.columns[col])
    print(classification_report(Y_test.iloc[:,col], Y_pred[:,col]))
    print('-----------------------------------------------------')

related
              precision    recall  f1-score   support

           0       0.67      0.04      0.08      4582
           1       0.77      0.99      0.86     14936
           2       0.24      0.11      0.15       144

    accuracy                           0.76     19662
   macro avg       0.56      0.38      0.37     19662
weighted avg       0.74      0.76      0.68     19662

-----------------------------------------------------
request
              precision    recall  f1-score   support

           0       0.91      0.96      0.93     16288
           1       0.73      0.52      0.61      3374

    accuracy                           0.88     19662
   macro avg       0.82      0.74      0.77     19662
weighted avg       0.88      0.88      0.88     19662

-----------------------------------------------------
offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     19570
           1       0.03      0.01      0.02        92

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     19577
           1       0.00      0.00      0.00        85

    accuracy                           0.99     19662
   macro avg       0.50      0.50      0.50     19662
weighted avg       0.99      0.99      0.99     19662

-----------------------------------------------------
aid_centers
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     19427
           1       0.09      0.04      0.05       235

    accuracy                           0.98     19662
   macro avg       0.54      0.52      0.52     19662
weighted avg       0.98      0.98      0.98     19662

-----------------------------------------------------
other_infrastructure
              precision    recall  f1-score   support

           0       0.96      0.99      0.97     18781
           1       0.24      0.10      0.14       881

    accuracy                           0.

In [19]:
print('Accuracy: {}'.format(np.mean(Y_test.values == Y_pred)))

Accuracy: 0.9437889216650278


### 9. Export your model as a pickle file

In [20]:
filename = 'classifier.sav'
pickle.dump(model, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.